In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install torch-geometric
!pip install torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing inst

In [3]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Download and load the training dataset
train_dataset = datasets.MNIST(root='data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Load the test dataset
test_dataset = datasets.MNIST(root='data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 57.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.61MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.8MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.69MB/s]


In [4]:
import torch
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn

class autoencoder(nn.Module):
    def __init__(self, x_dim, hidden_dim, z_dim=10):
        super(autoencoder, self).__init__()

        #Define autoencoding layers
        self.enc_layer1 = nn.Linear(x_dim, hidden_dim)
        self.enc_layer2 = nn.Linear(hidden_dim, z_dim)
        
        self.dec_layer1 = nn.Linear(z_dim, hidden_dim)
        self.dec_layer2 = nn.Linear(hidden_dim, x_dim)

    def encoder(self, x):
        x = F.relu(self.enc_layer1(x))
        z = F.relu(self.enc_layer2(x))
        return z
        
    def decoder(self, z):
        output = F.relu(self.dec_layer1(z))
        output = F.relu(self.dec_layer2(output))
        return output
        
    def forward(self, x):
        z = self.encoder(x)
        output = self.decoder(z)
        return output

def train_model(dataset, learning_rate=1e-3, batch_size=128, num_epochs=15):
    #X = torch.tensor(X).float()
    #dataset = TensorDataset(X)
    dataloader = DataLoader(dataset, batch_size= batch_size, shuffle=True)

    #Instantiate model and optimizer
    model = autoencoder(x_dim = 784, hidden_dim=400, z_dim=20)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    def loss_function(output, x):
        recon_loss = F.mse_loss(output, x, reduction='sum')
        return recon_loss
    
    #Train the model
    for epoch in range(num_epochs):
        epoch_loss = 0
        for batch in dataloader:
            
            optimizer.zero_grad()
            x = batch[0]
            x = x.view(-1, 784)
            output = model(x)
            loss = loss_function(output, x)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(dataset)}")
            
train_model(train_dataset)

Epoch 1/15, Loss: 725.627150390625
Epoch 2/15, Loss: 725.4731333333333
Epoch 3/15, Loss: 725.4730001302083
Epoch 4/15, Loss: 725.4728325520833
Epoch 5/15, Loss: 725.4733744791666
Epoch 6/15, Loss: 725.47304296875
Epoch 7/15, Loss: 725.4725704427084
Epoch 8/15, Loss: 725.4735174479167
Epoch 9/15, Loss: 725.4726645833333
Epoch 10/15, Loss: 725.4725970052083
Epoch 11/15, Loss: 725.4723401041666
Epoch 12/15, Loss: 725.4724356770834
Epoch 13/15, Loss: 725.4727817708333
Epoch 14/15, Loss: 725.47238515625
Epoch 15/15, Loss: 725.47245


In [5]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import torch.nn.functional as F

class VAE(nn.Module):
    def __init__(self, x_dim, hidden_dim, z_dim=10):
        super(VAE, self).__init__()
        self.enc_layer1 = nn.Linear(x_dim, hidden_dim)
        self.enc_layer2_mu = nn.Linear(hidden_dim, z_dim)
        self.enc_layer2_logvar = nn.Linear(hidden_dim, z_dim)
        
        self.dec_layer1 = nn.Linear(z_dim, hidden_dim)
        self.dec_layer2 = nn.Linear(hidden_dim, x_dim)

    def encoder(self, x):
        x = F.relu(self.enc_layer1(x))
        mu = F.relu(self.enc_layer2_mu(x))
        logvar = F.relu(self.enc_layer2_logvar(x))
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(logvar/2)
        eps = torch.randn_like(std)
        z = mu + logvar * eps
        
    def decoder(self, z):
        output = F.relu(self.dec_layer1(z))
        output = F.relu(self.dec_layer2(output))
        return x

    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        output = self.decoder(z)
        return output, z, mu, logvar

def loss_function(output, x, mu, logvar):
    recon_loss = F.mse_loss(output, x, reduction='sum') / batch_size
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + 0.002 * kl_loss

def train_model(X, learning_rate=1e-4, batch_size=128, num_epochs=15, hidden_dim=256, latent_dim=50):
    model = VAE(x_dim=X.shape[1], hidden_dim=hidden_dim, z_dim=latent_dim)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    X = torch.tensor(X).float()
    dataset = TensorDataset(X)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(num_epochs):
        epoch_loss = 0
        for batch in dataloader:
            optimizer.zero_grad()
            x = batch[0]
            output, z, mu, logvar = model(x)

            loss = loss_function(output, x, mu, logvar)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(X)}")
    return model